### Converting PDF to Text

In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
import PyPDF2

In [3]:
def pdf_to_text(pdf_path, output_txt):
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PdfReader object instead of PdfFileReader
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Initialize an empty string to store the text
        text = ''

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Write the extracted text to a text file
    with open(output_txt, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [4]:
if __name__ == "__main__":
    pdf_path = 'Recursion.pdf'

    output_txt = 'Recursion.txt'

    pdf_to_text(pdf_path, output_txt)

    print("PDF converted to text successfully!")

PDF converted to text successfully!


### Installation of Packages

In [5]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [6]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [7]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [8]:
pip install -U langchain-community

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
    --------------------------------------- 0.0/2.3 MB 653.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.3 MB 2.0 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.3 MB 3.2 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.3 MB 3.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.9/2.3 MB 3.4 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.3 MB 3.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.4/2.3 MB 4.2 MB/s eta 0:00:01
   ---------------------------- ----------- 1.7/2.3 MB 4.3 MB/s eta 0:00:01
   ---------------------------------- ----- 2.0/2.3 MB 4.5 MB/s eta 0:00:01
   -------------------------------------- - 2.2/2.3 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 4.5 MB/s eta 0:00:00
   ---------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [9]:
import os
import chromadb
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Get the Embedding Model

In [10]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Embedding Model Loaded..........")

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Model Loaded..........


### Data Preprocessing

In [11]:
from langchain_community.document_loaders import TextLoader

class CustomTextLoader(TextLoader):
    def __init__(self, file_path: str, encoding: str = 'utf-8'):
        super().__init__(file_path)
        self.encoding = encoding

    def lazy_load(self):
        try:
            with open(self.file_path, encoding=self.encoding) as f:
                text = f.read()
            return [text]
        except UnicodeDecodeError as e:
            raise RuntimeError(f"Error loading {self.file_path}") from e
            
def process_text_file(text_filename, encoding='utf-8'):
    # Load text from the text file
    loader = TextLoader(text_filename, encoding)
    text_content = loader.load()

    # Split text using RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(text_content)

    return text_chunks

# Example usage
text_filename = 'Recursion.txt'
documents = process_text_file(text_filename)

### Create and Store Vectors, Load Vector Store

In [12]:
recursion_store = Chroma.from_documents(documents, hf, collection_metadata={"hnsw:space": "cosine"}, persist_directory="recursion_chroma_cosine")

In [13]:
import shutil

# Source directory to be zipped
source_dir = r'recursion_chroma_cosine'
# Destination zip file path
zip_file_path = r'recursion_chroma_cosine.zip'

# Create a zip file
shutil.make_archive(zip_file_path, 'zip', source_dir)

print(f"Directory {source_dir} zipped successfully into {zip_file_path}.zip")

Directory recursion_chroma_cosine zipped successfully into recursion_chroma_cosine.zip.zip


In [14]:
load_recursion_store = Chroma(persist_directory="recursion_chroma_cosine", embedding_function=hf)

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [15]:
retriever_recursion = load_recursion_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [16]:
lotr = MergerRetriever(retrievers=[retriever_recursion])

### Perform Semantic Search

In [17]:
query = "What is the running time of Merge Sort Algorithm?"

In [18]:
docs = lotr.get_relevant_documents(query)
docs

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'Recursion.txt'}, page_content='™Ignoring Floors and Ceilings Is Okay, Honest\nCareful readers might object that our analysis brushes an important detail under\nthe rug. The running time of mergesort doesn’t really obey the recurrence\nT(n)=2T(n/2)+O(n); after all, the input size nmight be odd, and what could\nit possibly mean to sort an array of size 421\n2or177\n8? The actual mergesort\nrecurrence is somewhat messier:\nT(n)=T\x00\ndn/2e\x00\n+T\x00\nbn/2c\x00\n+O(n).\nSure, we could check that T(n)=O(nlogn)using induction, but the necessary\ncalculations would be awful. Fortunately, there is a simple technique for\nremoving ﬂoors and ceilings from recurrences, called domain transformation .\n•First, because we are deriving an upper bound, we can safely overestimate\nT(n), once by pretending that the two subproblem sizes are equal, and\nagain to eliminate the ceiling:\x00\nT(n)\uf8ff2T\x00\ndn/2e\x00\n+n\uf8ff2T(n/2+1)+n.\n\x00Formally , we are treating T

In [19]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(metadata={'source': 'Recursion.txt'}, page_content='™Ignoring Floors and Ceilings Is Okay, Honest\nCareful readers might object that our analysis brushes an important detail under\nthe rug. The running time of mergesort doesn’t really obey the recurrence\nT(n)=2T(n/2)+O(n); after all, the input size nmight be odd, and what could\nit possibly mean to sort an array of size 421\n2or177\n8? The actual mergesort\nrecurrence is somewhat messier:\nT(n)=T\x00\ndn/2e\x00\n+T\x00\nbn/2c\x00\n+O(n).\nSure, we could check that T(n)=O(nlogn)using induction, but the necessary\ncalculations would be awful. Fortunately, there is a simple technique for\nremoving ﬂoors and ceilings from recurrences, called domain transformation .\n•First, because we are deriving an upper bound, we can safely overestimate\nT(n), once by pretending that the two subproblem sizes are equal, and\nagain to eliminate the ceiling:\x00\nT(n)\uf8ff2T\x00\ndn/2e\x00\n+n\uf8ff2T(n/2+1)+n.\n\x00Formally , we are treating T

In [20]:
query2 = "What is the fastest multiplication algorithm and who found it?"

In [21]:
docs = lotr.get_relevant_documents(query2)
docs

[Document(metadata={'source': 'Recursion.txt'}, page_content='discovery of the Fast Fourier transform .\x00\x00The basic FFT algorithm itself\nruns in O(nlogn)time; however, using FFTs for integer multiplication incurs\nsome small additional overhead. The ﬁrst FFT-based integer multiplication\nalgorithm, published by Arnold Schönhage and Volker Strassen in \x00\x00\x00\x00, runs\ninO(nlognlog log n)time. Schönhage-Strassen remained the theoretically\nfastest integer multiplication algorithm for several decades, before Martin Fürer\ndiscovered the ﬁrst of a long series of technical improvements. Finally , in \x00\x00\x00\x00,\nDavid Harvey and Joris van der Hoeven published an algorithm that runs in\nO(nlogn)time.\x00\x00\n\uffff.\uffff\uffff Exponentiation\nGiven a number aand a positive integer n, suppose we want to compute an. The\nstandard naïve method is a simple for-loop that performs n\x001multiplications\nbya:\n\x00\x00Seehttp://algorithms.wtf for lecture notes on Fast Fourier t

In [22]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(metadata={'source': 'Recursion.txt'}, page_content='discovery of the Fast Fourier transform .\x00\x00The basic FFT algorithm itself\nruns in O(nlogn)time; however, using FFTs for integer multiplication incurs\nsome small additional overhead. The ﬁrst FFT-based integer multiplication\nalgorithm, published by Arnold Schönhage and Volker Strassen in \x00\x00\x00\x00, runs\ninO(nlognlog log n)time. Schönhage-Strassen remained the theoretically\nfastest integer multiplication algorithm for several decades, before Martin Fürer\ndiscovered the ﬁrst of a long series of technical improvements. Finally , in \x00\x00\x00\x00,\nDavid Harvey and Joris van der Hoeven published an algorithm that runs in\nO(nlogn)time.\x00\x00\n\uffff.\uffff\uffff Exponentiation\nGiven a number aand a positive integer n, suppose we want to compute an. The\nstandard naïve method is a simple for-loop that performs n\x001multiplications\nbya:\n\x00\x00Seehttp://algorithms.wtf for lecture notes on Fast Fourier t

### List of Questions and Context

In [23]:
questions = [
    "What is the running time of Merge Sort Algorithm?",
    "What is the running time of the fastest multiplication algorithm and who found it?",
    "Explain the steps involved in the quicksort algorithm, and how does it work?",
    "What is the worst case and average case time complexity of quick sort algorithm?",
    "What is the time complexity of multiplying two integers using Karatsuba algorithm?",
    "What is the time complexity for the exponentiation problem using the method proposed by Pingala?",
    "Suppose you are given a stack of n pancakes of different sizes. You want to sort the pancakes so that smaller pancakes are on top of larger pancakes. The only operation you can perform is a flip—insert a spatula under the top k pancakes, for some integer k between 1 and n, and flip them all over. Describe an algorithm to sort an arbitrary stack of n pancakes using O(n) flips",
    "Prove the Correctness of Mergesort Algorithm."
]

In [24]:
docs = [lotr.get_relevant_documents(query) for query in questions]
docs_contents = []
for x in docs:
    contents = [y.page_content for y in x]
    docs_contents.append(contents)

In [25]:
file = open("proof.txt")
proof = file.read()

In [26]:
ground_truths = [
    "The running time of Merge Sort Algorithm is O(n log n).",
    "The fastest multiplication algorithm runs in O(n log n) time and was found out by David Harvey and Joris van der Hoeven.",
    "Quicksort is a recursive sorting algorithm that operates in three main steps:\n 1. Choose a Pivot: Select a pivot element from the array. This element will be used to divide the array into subarrays.\n 2. Partition: Rearrange the elements in the array such that all elements smaller than the pivot come before it, the pivot element itself is in the middle, and all elements larger than the pivot come after it. This results in three subarrays: one with elements less than the pivot, one with the pivot itself, and one with elements greater than the pivot. \n 3. Recursively Apply Quicksort: Apply the quicksort algorithm recursively to the subarrays containing elements smaller and larger than the pivot.By focusing on partitioning the array efficiently, quicksort ensures that sorting the subarrays becomes straightforward and manageable.",
    "The worst-case time complexity of quicksort is O(n^2). The average-case time complexity of quicksort is O(nlogn).",
    "The time complexity of Karatsuba algorithm is O(n^(log 3 base 2)), which is approximately O(n^1.585).",
    "The time complexity of Pingala's method is O(log n) where n is the exponent.",
    "Start from the unsorted portion of the stack, which initially includes all pancakes. Identify the largest pancake in this portion. If the largest pancake is not already at the top, flip the stack up to the position of the largest pancake to bring it to the top. Flip the stack up to the position where the largest pancake should be (which is the current end of the unsorted portion of the stack). This places the largest pancake at its correct position at the bottom of the unsorted portion. Consider the sorted portion (which now includes the largest pancake at the bottom) and repeat the process for the remaining unsorted pancakes. Recursively the algorithm will sort the stack of pancakes.",
    proof]

### Generating Answers

In [27]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [28]:
pip install langchain_groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [29]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

#### Using llama3-8b-8192

In [30]:
chat_model = ChatGroq(temperature=0,
              model_name="llama3-8b-8192",
              api_key="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")

In [31]:
chain = prompt | chat_model

In [32]:
ai_messages_1 = [chain.invoke({"context":docs[i],"question":questions[i]}) for i in range(4)]
answers_1 = [ai_message.content for ai_message in ai_messages_1]

In [33]:
query1 = "What is the running time of Merge Sort Algorithm?"
docs1 = lotr.get_relevant_documents(query1)
chain.invoke({"context":docs1,"question":query1}).content

'According to the provided context, the running time of the Merge Sort algorithm is O(n log n).'

#### Using mixtral

In [34]:
chat_model = ChatGroq(temperature=0,
              model_name="mixtral-8x7b-32768",
              api_key="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")

In [35]:
chain = prompt | chat_model

In [36]:
ai_messages_2 = [chain.invoke({"context":docs[i],"question":questions[i]}) for i in range(4)]
answers_2 = [ai_message.content for ai_message in ai_messages_2]

In [37]:
query2 = "What is the running time of the fastest multiplication algorithm and who found it?"
docs2 = lotr.get_relevant_documents(query2)
chain.invoke({"context":docs2,"question":query2}).content

'The fastest multiplication algorithm runs in O(nlogn) time and it was found by David Harvey and Joris van der Hoeven.'

### Evaluating RAG pipeline

In [38]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [39]:
import pandas as pd

In [40]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [41]:
from datasets import Dataset

In [42]:
details = {'question': questions[:2], 'answer': answers_1[:2], 'contexts':docs_contents[:2], 'ground_truth':ground_truths[:2]}
dataset = Dataset.from_dict(details)

In [43]:
dataset1 = dataset.to_pandas()
dataset1

,question,answer,contexts,ground_truth
0,What is the running time of Merge Sort Algorithm?,"According to the provided context, the running...","[™Ignoring Floors and Ceilings Is Okay, Honest...",The running time of Merge Sort Algorithm is O(...
1,What is the running time of the fastest multip...,The running time of the fastest multiplication...,[discovery of the Fast Fourier transform .  Th...,The fastest multiplication algorithm runs in O...


In [44]:
pip install ragas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [45]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-r6xO8hN3apVazpBUR9m8T3BlbkFJkwY5u5AxMi36taj6apRV"

In [46]:
pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [47]:
import nest_asyncio
nest_asyncio.apply()

In [48]:
from ragas.metrics import (
    answer_similarity,
    answer_relevancy,
    answer_correctness,
    faithfulness,
    context_recall,
    context_precision,
)

In [49]:
metrics = [
    answer_similarity,
]

In [50]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
llm = ChatGroq(temperature=0,
              model_name="llama3-70b-8192",
              api_key="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")
ragas_groq_llm = LangchainLLMWrapper(llm)

In [51]:
for m in metrics:
    # change LLM for metric
    m.__setattr__("llm", ragas_groq_llm)

In [52]:
embeddings = LangchainEmbeddingsWrapper(hf)

In [53]:
if hasattr(m, "embeddings"):
        # if so change with VertexAI Embeddings
        m.__setattr__("embeddings", embeddings)

In [54]:
from ragas import evaluate
evaluate_result = evaluate(
        dataset=dataset,
        metrics=metrics,
        raise_exceptions = False, 
)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [55]:
answer_similarities = evaluate_result.to_pandas()
answer_similarities

,question,answer,contexts,ground_truth,answer_similarity
0,What is the running time of Merge Sort Algorithm?,"According to the provided context, the running...","[™Ignoring Floors and Ceilings Is Okay, Honest...",The running time of Merge Sort Algorithm is O(...,0.977050
1,What is the running time of the fastest multip...,The running time of the fastest multiplication...,[discovery of the Fast Fourier transform .  Th...,The fastest multiplication algorithm runs in O...,0.972157
